#### Projeto: Simulação de Medições de Glicose
O projeto visa realizar modelagem preditiva com o modelo LSTM aplicado em séries temporais.

https://colab.research.google.com/drive/1tcbR5JI2FLDDU49JkyarDPyzrE2ly2KR#scrollTo=iffWW7CGpuI3

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# 1. Dados simulados
np.random.seed(42)
n = 300
base_time = pd.Timestamp("2023-01-01 00:00:00")
time_deltas = np.cumsum(np.random.randint(30, 180, size=n))  # em minutos
timestamps = [base_time + timedelta(minutes=int(m)) for m in time_deltas]

In [ ]:
# Simular variação de glicose (base + ruído + padrão diário)
glucose = 90 + 15 * np.sin(np.array([t.hour for t in timestamps]) / 24 * 2 * np.pi) + np.random.normal(0, 10, n)

df = pd.DataFrame({"timestamp": timestamps, "glucose": glucose})
df["delta_t"] = df["timestamp"].diff().dt.total_seconds().fillna(0)

In [ ]:
# 2. Normalizar e criar sequências para LSTM
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["glucose", "delta_t"]])

sequence_length = 10
X, y = [], []

for i in range(sequence_length, len(df_scaled)):
    X.append(df_scaled[i-sequence_length:i])
    y.append(df_scaled[i, 0])  # previsão da glicose

X, y = np.array(X), np.array(y)

In [ ]:
# 3. Modelo LSTM
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=20, batch_size=16, verbose=0)

In [ ]:
# 4. Previsões
y_pred = model.predict(X)
glucose_pred = scaler.inverse_transform(np.concatenate([y_pred, X[:, -1, 1].reshape(-1,1)], axis=1))[:,0]
glucose_real = df["glucose"].values[sequence_length:]

In [ ]:
# 5. Visualizar
plt.figure(figsize=(12, 6))
plt.plot(df["timestamp"].values[sequence_length:], glucose_real, label="Real", marker='o', alpha=0.6)
plt.plot(df["timestamp"].values[sequence_length:], glucose_pred, label="Previsto", marker='x', alpha=0.8)
plt.title("Previsão de Glicose com LSTM (tempo irregular)")
plt.xlabel("Tempo")
plt.ylabel("Glicose (mg/dL)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()